<a href="https://colab.research.google.com/github/MevrouwHelderder/final_assignment/blob/main/Final_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is your final assignment. You'll get a lot of freedom in doing this assignment but that also means you have to make choices and explain the reasoning behind those choices in your report.

For this assignment you can use any dataset you can find from the [Our World in Data website](https://ourworldindata.org/).

Please formulate an answer to the following three questions in your report.

* **What is the biggest predictor of a large CO2 output per capita of a country?**
* **which countries are making the biggest strides in decreasing CO2 output?**
* **which non-fossil fuel energy technology will have the best price in the future?**




---


**1: Biggest predictor of CO2 output**

To determine this you may want to consider things like GDP per capita, diets, number of cars per capita, various energy source, mobility and other factors.

Your answer can also be a specific combination of certain factors.


---


**2: Biggest strides in decreasing CO2 output**

You'll need to find the relative CO2 output for each country to be able to calculate this. But countries can have growing and shrinking populations too, so it's probably a good idea to take this into account as well.


---


**3: Best future price for non-fossil fuel energy**

To be able to predict prices you'll probably need to use linear regression over the various non-fossil fuel options.


---


**Submitting your Assignment**

Once you're done with this module, you can go to the next item where you'll be able to submit your assignment.

Please submit both your written report and all notebooks you've created in creating the report. Make sure everything works before submitting.



#Thoughts: 

Find what countries do: 
* did they make any pledges
* what kind of energy are they using now


What contributes to CO2 output?

What are the prices for the different types of non-fossil energy?

What happens when a country outsources everything?

How about CO2 vs the other GHG (greenhouse gasses)? Are there countries where CO2 falls but the other rise?


# What contributes to CO2 output?


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%load_ext google.colab.data_table

In [36]:
path_base = "https://raw.githubusercontent.com/MevrouwHelderder/final_assignment/main/"

paths = {
    "co2_greenhouse" : path_base + "owid-co2-data.csv",
    "energy_total" : path_base + "owid-energy-data.csv",
    "net_zero_pledge" : path_base + "net-zero-targets.csv",
    "agri_land_use_total" : path_base + "total-agricultural-area-over-the-long-term.csv",
    "grazing" : path_base + "grazing-land-use-over-the-long-term.csv",
    "cropland" : path_base + "cropland-use-over-the-long-term.csv"
}

dataframes = {}

for key, value in paths.items():
  dataframes[key] = pd.read_csv(value)

energy_total_df = dataframes["energy_total"]
co2_greenhouse_df = dataframes["co2_greenhouse"]
net_zero_pledge_df = dataframes["net_zero_pledge"]
agri_land_use_total_df = dataframes["agri_land_use_total"]
grazing_df = dataframes["grazing"]
cropland_df = dataframes["cropland"]

# CO2 output

First let's look at what countries produces, import, export and how that changed through the years.

Note for later: 
also interesting: 
* what do we see when we look at rich vs poor (columns regarding GDP)
* what do we see when we look at the CO2 per energy unit.

In [ ]:
# co2: annual total production-based co2 emission. measured in million tonnes. Measured in million tonnes.
# co2_growth_abs: annual growth of production-based co2 emission. Measured in million tonnes.
# co2_growth_prct: annual percentage growth of production-based co2 emission. measured in million tonnes. Measured in million tonnes.
# co2_per_capita, annual total production-based emissions of carbon dioxide (CO₂), measured in tonnes per person. 

# trade_co2: annual net carbon dioxide (CO₂) emissions embedded in trade. The net of import or export via traded goods. Positive = importer of CO₂ emissions; Negative =  exporter. Measured in million tonnes.
# consumption_co2: total co2 minus emissions embedded in exports, plus emissions embedded in imports. Consumption > production = importer of CO₂ emissions; Consumption < Production =  exporter. Measured in million tonnes.

In [58]:
explore_co2 = co2_greenhouse_df.loc[:, ["country", "year", "iso_code", "population", "co2", "co2_growth_abs", "co2_growth_prct", "co2_per_capita", "trade_co2", "consumption_co2",  ]]
explore_co2 = explore_co2.set_index("year")
explore_co2.index = pd.to_datetime(explore_co2.index, format="%Y", errors="coerce")

# show only the true countries, not the combinations like continents
explore_co2 = explore_co2[~explore_co2["iso_code"].isnull()]


In [59]:
explore_co2

,country,iso_code,population,co2,co2_growth_abs,co2_growth_prct,co2_per_capita,trade_co2,consumption_co2
year,,,,,,,,,
1850-01-01,Afghanistan,AFG,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN
1851-01-01,Afghanistan,AFG,3769828.0,NaN,NaN,NaN,NaN,NaN,NaN
1852-01-01,Afghanistan,AFG,3787706.0,NaN,NaN,NaN,NaN,NaN,NaN
1853-01-01,Afghanistan,AFG,3806634.0,NaN,NaN,NaN,NaN,NaN,NaN
1854-01-01,Afghanistan,AFG,3825655.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2017-01-01,Zimbabwe,ZWE,14751101.0,9.596,-0.937,-8.899,0.651,0.910,10.506
2018-01-01,Zimbabwe,ZWE,15052191.0,11.795,2.199,22.920,0.784,0.771,12.567
2019-01-01,Zimbabwe,ZWE,15354606.0,11.115,-0.681,-5.772,0.724,0.978,12.092


In [60]:
germany_test = explore_co2[explore_co2 ["country"]=="Germany"]
germany_test

,country,iso_code,population,co2,co2_growth_abs,co2_growth_prct,co2_per_capita,trade_co2,consumption_co2
year,,,,,,,,,
1792-01-01,Germany,DEU,NaN,0.469,NaN,NaN,NaN,NaN,NaN
1793-01-01,Germany,DEU,NaN,0.480,0.011,2.344,NaN,NaN,NaN
1794-01-01,Germany,DEU,NaN,0.443,-0.037,-7.634,NaN,NaN,NaN
1795-01-01,Germany,DEU,NaN,0.447,0.004,0.826,NaN,NaN,NaN
1796-01-01,Germany,DEU,NaN,0.535,0.088,19.672,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2017-01-01,Germany,DEU,82624368.0,785.616,-14.723,-1.840,9.508,113.953,899.570
2018-01-01,Germany,DEU,82896696.0,754.408,-31.208,-3.972,9.101,130.370,884.778
2019-01-01,Germany,DEU,83148144.0,707.150,-47.258,-6.264,8.505,135.700,842.850
